In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head(1)

,acousticness,artist_name,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,title,popularity,release_date,speechiness,tempo,valence,year
0,0.995,Carl Woitschach,0.708,158648,0.195,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.151,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.779,1928


In [4]:
df = df[["id","title", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence","popularity"]]

In [5]:
df.shape

(169909, 15)

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
X_wnames = np.array(df['title'])
X = np.array(df.drop(['id','popularity','title'], axis = 1))
y = np.array(df['popularity'])

In [7]:
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
pca = decomposition.PCA(n_components=8)
X_scaled = StandardScaler().fit_transform(X)
X_pca = pca.fit_transform(X_scaled)

In [8]:
from scipy.sparse import csr_matrix, hstack
X_final = csr_matrix(X_pca)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2)

In [10]:
from sklearn.neighbors import NearestNeighbors

knn_params = {'n_neighbors':range(1,10)}
knn = NearestNeighbors(n_jobs=-1, n_neighbors=11)
knn.fit(X_pca)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=11, p=2,
                 radius=1.0)

In [24]:
X_pcadf = pd.DataFrame(X_pca)
query_index=np.random.choice(X_wnames.shape[0])
distances, indices = knn.kneighbors(X_pcadf.iloc[query_index,:].values.reshape(1,-1), n_neighbors =11)
for i in range(0,len(distances.flatten())):
    if i==0:
        print("Recommendations for {0}:\n".format(X_wnames[query_index]))
    else:
        print('{0}: {1}, with distance of {2}'.format(i, X_wnames[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Things I Can't Change:

1: Head Club, with distance of 0.5847750395046579
2: More or Less, with distance of 0.6474652171441817
3: Deadbolt, with distance of 0.6943955096867742
4: How About You, with distance of 0.719054918477085
5: Hyper Music, with distance of 0.7199137321348987
6: Let Forever Be, with distance of 0.7216250975089511
7: Trade Mistakes, with distance of 0.759473342768027
8: Free (with Drew Love) - Party Favor Remix, with distance of 0.7693480078918741
9: What Lies Beneath, with distance of 0.7775268404080933
10: Hear Me Now, with distance of 0.7945041356683352
